In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
import tensorflow_hub as hub

import os
import random
import shutil
from math import ceil, sqrt

In [17]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [18]:

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [19]:

!kaggle datasets download -d puneet6060/intel-image-classification

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
 96% 333M/346M [00:07<00:00, 37.6MB/s]
100% 346M/346M [00:07<00:00, 49.8MB/s]


In [20]:
!ls

 intel-image-classification.zip  'kaggle (1).json'   kaggle.json   sample_data


In [ ]:
!unzip intel-image-classification.zip

In [23]:
!ls

 intel-image-classification.zip   kaggle.json   seg_pred   seg_train
'kaggle (1).json'		  sample_data   seg_test


In [24]:
#INPUT_DIR = "intel-image-classification/"
TRAIN_DIR ="seg_train/seg_train"
TEST_DIR = "seg_test/seg_test"
PRED_DIR = "seg_pred/seg_pred"

In [ ]:
# Define the image file paths
images = [
    TRAIN_DIR + "/sea/10016.jpg", TRAIN_DIR + "/sea/10041.jpg",
    TRAIN_DIR + "/buildings/10006.jpg", TRAIN_DIR + "/buildings/10014.jpg",
    TRAIN_DIR + "/forest/10007.jpg", TRAIN_DIR + "/forest/10010.jpg",
    TRAIN_DIR + "/glacier/10003.jpg", TRAIN_DIR + "/glacier/10009.jpg",
    TRAIN_DIR + "/mountain/10000.jpg", TRAIN_DIR + "/mountain/10002.jpg",
    TRAIN_DIR + "/street/10015.jpg", TRAIN_DIR + "/street/10019.jpg"
]

# Define the corresponding categories for each image
categories = [image.split("/")[-2] for image in images]

plt.figure(figsize=(18, 12))
for i in range(12):
    img = plt.imread(images[i])
    plt.subplot(3, 4, i + 1)
    plt.imshow(img)
    plt.title(categories[i])  # Display the category in the title
    plt.axis("off")
plt.show()

In [26]:
# Create the training and validation directories if they don't exist
train_dir = 'train'
valid_dir = 'val'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

for subdir in os.listdir(TRAIN_DIR):
    subdir_path = os.path.join(TRAIN_DIR, subdir)

    # Create corresponding subdirectories in train and val directories
    train_subdir = os.path.join(train_dir, subdir)
    valid_subdir = os.path.join(valid_dir, subdir)

    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(valid_subdir, exist_ok=True)

    all_images = os.listdir(subdir_path)

    num_validation = int(len(all_images) * 0.2)
    validation_images = random.sample(all_images, num_validation)

    for img in all_images:
        source_path = os.path.join(subdir_path, img)
        dest_dir = valid_subdir if img in validation_images else train_subdir
        dest_path = os.path.join(dest_dir, img)
        shutil.copy(source_path, dest_path)

In [28]:
!ls

 intel-image-classification.zip   kaggle.json   seg_pred   seg_train   val
'kaggle (1).json'		  sample_data   seg_test   train


In [29]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,      # Rescale pixel values to [0, 1]
    rotation_range=25,      # Randomly rotate images by up to 25 degrees
    width_shift_range=0.3,  # Randomly shift the width of images
    height_shift_range=0.3, # Randomly shift the height of images
    horizontal_flip=True,   # Randomly flip images horizontally
    shear_range=0.3,        # Apply shear transformations
    zoom_range=0.4,         # Randomly zoom into images
    fill_mode='nearest'     # Fill empty pixels with the nearest value
)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
# Create data generators for training and validation data
batch_size = 32
image_size = (150, 150)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    'val',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=image_size,
    shuffle=False,
    class_mode='categorical',
    batch_size=1
)

Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [31]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

image_size = (224, 224)  # example size, replace accordingly

efficient_net = hub.KerasLayer(
    "https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet21k-b3-feature-vector/versions/1",
    trainable=False, input_shape=image_size + (3,)
)

model = keras.Sequential([
    layers.Lambda(efficient_net),  # Wrap hub layer with Lambda
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(6, activation='softmax'),
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
early_stopping = keras.callbacks.EarlyStopping( monitor='val_accuracy',patience=5,restore_best_weights=True)

In [33]:
history = model.fit(train_generator, epochs=5, validation_data=val_generator, callbacks=[early_stopping])

Epoch 1/5
351/351 ━━━━━━━━━━━━━━━━━━━━ 732s 2s/step - accuracy: 0.8087 - loss: 0.5208 - val_accuracy: 0.9116 - val_loss: 0.2605
Epoch 2/5
351/351 ━━━━━━━━━━━━━━━━━━━━ 736s 2s/step - accuracy: 0.8324 - loss: 0.4535 - val_accuracy: 0.9183 - val_loss: 0.2345
Epoch 3/5
351/351 ━━━━━━━━━━━━━━━━━━━━ 748s 2s/step - accuracy: 0.8430 - loss: 0.4260 - val_accuracy: 0.9076 - val_loss: 0.2533
Epoch 4/5
351/351 ━━━━━━━━━━━━━━━━━━━━ 734s 2s/step - accuracy: 0.8532 - loss: 0.3986 - val_accuracy: 0.9165 - val_loss: 0.2393
Epoch 5/5
351/351 ━━━━━━━━━━━━━━━━━━━━ 723s 2s/step - accuracy: 0.8644 - loss: 0.3763 - val_accuracy: 0.9126 - val_loss: 0.2356


In [34]:
predictions = model.predict(test_generator)
loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.4f}, accuracy: {accuracy:.4f}')

3000/3000 ━━━━━━━━━━━━━━━━━━━━ 212s 70ms/step
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 215s 72ms/step - accuracy: 0.9258 - loss: 0.2176
Test loss: 0.2298, accuracy: 0.9237


In [ ]:
true_labels = test_generator.classes
class_names = list(test_generator.class_indices.keys())

# Calculate the confusion matrix
confusion = confusion_matrix(true_labels, np.argmax(predictions, axis=-1))

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
misclassified_indices = np.where(np.argmax(predictions, axis=-1) != true_labels)[0]
np.random.shuffle(misclassified_indices)

num_images = min(len(misclassified_indices), 20)
num_cols = int(ceil(sqrt(num_images)))
num_rows = int(ceil(num_images / num_cols))

fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 12))

for i, idx in enumerate(misclassified_indices[:num_images]):
    true_class, predicted_class = true_labels[idx], np.argmax(predictions[idx])
    misclassified_image = plt.imread(test_generator.filepaths[idx])
    axs[i // num_cols, i % num_cols].imshow(misclassified_image)
    axs[i // num_cols, i % num_cols].set_title(f'True: {list(test_generator.class_indices.keys())[true_class]}\nPredicted: {list(test_generator.class_indices.keys())[predicted_class]}')

for i in range(num_images, num_rows * num_cols):
    axs[i // num_cols, i % num_cols].axis('off')

fig.suptitle("Mislassified images")
plt.tight_layout()
plt.show()